In [11]:
import Bio

from Bio.Blast.Applications import NcbiblastxCommandline
from Bio.Blast.Applications import NcbiblastpCommandline

cline = NcbiblastpCommandline(query="rosemary.pro", db="nr",
                              evalue=0.001, remote=True, ungapped=True)
blastx_cline = NcbiblastxCommandline(query="yakuba.fa", db="nr", evalue=0.001,outfmt=5, out="test.xml")
blastx_cline

print(blastx_cline)

FileNotFoundError: [Errno 2] No such file or directory: 'test.xml'

In [2]:
#test function to make sure Biopython and blast both work

import Bio

from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastn", "nt", "8332116")

from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)
for blast_record in blast_records:
    print(blast_record.database_sequences)
    # Do something with blast_record

90704180


In [28]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio import Entrez
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET
import time

Entrez.email = 'ktrang@ufl.edu'

path = "/home/ktrang/blue_ferrallm/ktrang/molerats/fasta_genomes"

# Define a list of HomoloGene IDs and a dictionary of genome file names
#homologene_ids = ["1261", "55992", "1171"]
homologene_ids = [["ARNT","1261"], ["COPS5","55992"], ["HIF1A","1171"]]
#genome_files = {"species1": directory + "GCA_000001895.4_Rnor_6.0_genomic.fasta", "species2": "genome2.fasta"}
genome_files = [f for f in listdir(path) if isfile(join(path, f))]

# Loop over the HomoloGene IDs
for homologene_id in homologene_ids:
    # Load the HomoloGene record from NCBI
    handle = Entrez.efetch(db="homologene", id=homologene_id[1], retmode="xml")
    record = ET.parse(handle)
    #root = tree.getroot() #method to retrieve specific data from xml
    #record = Entrez.read(handle)[0] #depreciated entrez read for ncbi xml format, does not work with binary open
    time.sleep(.5)
    
    # Find geneID of human gene 
#     handle = Entrez.esearch(db="gene", term=homologene_id[0]+"[Gene] AND human[orgn]")
#     #handle = Entrez.esearch(db="nucleotide", term=homologene_id[0]+"[nt] AND human[orgn]")
#     record = Entrez.read(handle)
#     geneID = record["IdList"][0]
#     print(geneID)
    gene_id=405
    time.sleep(.5)
    
    # Extract the query sequence for the human gene in the HomoloGene record
    handle = Entrez.esummary(db="gene", id=gene_id)
    record = Entrez.read(handle)
    # Get the first GenBank accession number
    assembly_id = record["DocumentSummarySet"]["DocumentSummary"][0]["LocationHist"][0]["AssemblyAccVer"]
    time.sleep(.5)

    # Use E-utilities to retrieve the nucleotide sequence
    handle = Entrez.efetch(db="nuccore", id=assembly_id, rettype="fasta", retmode="text")
    record = SeqIO.read(handle, "fasta")

    # Print the nucleotide sequence
    print(record.seq)
    
    #human_seq = SeqRecord(Seq(record["homologene_member"]["0"]["sequence"]["@value"]), id="human_gene")

    # Loop over the genome files
    for species, genome_file in genome_files.items():
        # Load the genome sequence for the species of interest from a saved file
        genome_seq = SeqIO.read(path + genome_file, "fasta")

        # Run a BLAST search
        result_handle = NCBIWWW.qblast("blastn", "nt", human_seq, expect=0.01)

        # Parse the BLAST search results
        blast_records = NCBIXML.parse(result_handle)

        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    # Extract the gene ID from the hit location in the genome
                    hit_start = min(hsp.sbjct_start, hsp.sbjct_end)
                    hit_end = max(hsp.sbjct_start, hsp.sbjct_end)
                    hit_id = genome_seq.id + ":" + str(hit_start) + "-" + str(hit_end)
                    
                    # Print the gene ID and alignment information
                    print("HomoloGene ID:", homologene_id)
                    print("Species:", species)
                    print("Query:", blast_record.query)
                    print("Hit ID:", hit_id)
                    print("Alignment length:", hsp.align_length)
                    print("Percent identity:", hsp.identities / hsp.align_length)
                    print("E-value:", hsp.expect)
                    print("Bit score:", hsp.bits)
                    print()

HTTPError: HTTP Error 400: Bad Request

In [3]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio import Entrez
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET

Entrez.email = 'ktrang@ufl.edu'
# Define your Gene ID
gene_id = "405"

# Use E-utilities to retrieve gene information
handle = Entrez.esummary(db="gene", id=gene_id)
record = Entrez.read(handle)

# Get the first GenBank accession number
assembly_id = record["DocumentSummarySet"]["DocumentSummary"][0]["LocationHist"][0]["AssemblyAccVer"]

print(assembly_id)


GCF_000001405.40


In [4]:
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# create some sample sequences
seq1 = Seq("ATCGTAGCTAGCTAGCTG")
seq2 = Seq("ATCGTGGCTAGTTAGCTG")
seq3 = Seq("ATCGTGGCTAGCTAGTTG")

# create SeqRecord objects for each sequence
rec1 = SeqRecord(seq1, id="Seq1")
rec2 = SeqRecord(seq2, id="Seq2")
rec3 = SeqRecord(seq3, id="Seq3")

# create a MultipleSeqAlignment object
alignments = MultipleSeqAlignment([rec1, rec2, rec3])

# perform the alignment using the default algorithm (ClustalW)
alignments_clustalw = alignments.align()

# print the resulting alignment
print(alignments_clustalw)

AttributeError: 'MultipleSeqAlignment' object has no attribute 'align'

In [5]:
from Bio.Align.Applications import ClustalwCommandline
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import subprocess

# create some sample sequences
seq1 = Seq("ATCGTAGCTAGCTAGCTG")
seq2 = Seq("ATCGTGGCTAGTTAGCTG")
seq3 = Seq("ATCGTGGCTAGCTAGTTG")

# create SeqRecord objects for each sequence
rec1 = SeqRecord(seq1, id="Seq1")
rec2 = SeqRecord(seq2, id="Seq2")
rec3 = SeqRecord(seq3, id="Seq3")

# create a MultipleSeqAlignment object
alignments = MultipleSeqAlignment([rec1, rec2, rec3])

path = "/home/ktrang/blue_ferrallm/ktrang/molerats"

cline = ClustalwCommandline("clustalw2", infile=alignments)

subprocess.call(str(cline))
#align = AlignIO.read(path + "/genes/" + "ARNT.fasta", "clustal")
#print(align)

FileNotFoundError: [Errno 2] No such file or directory: 'clustalw2 -infile=Alignment with 3 rows and 18 columns\nATCGTAGCTAGCTAGCTG Seq1\nATCGTGGCTAGTTAGCTG Seq2\nATCGTGGCTAGCTAGTTG Seq3'

In [ ]:
from Bio.Align.Applications import ClustalwCommandline
from Bio import AlignIO
import subprocess

path = "/home/ktrang/blue_ferrallm/ktrang/molerats/fasta_genomes"

# Define the input and output file names
input_file = path+ "/" + "Fukomys_damarensis.DMR_v1.0.dna.toplevel.fasta"
output_file = "aligned_sequences.aln"

# Create the ClustalW command-line object
clustalw_cline = ClustalwCommandline("/apps/clustalw/2.1/bin/clustalw2", infile=input_file)
print(clustalw_cline)
subprocess(clustalw_cline())

# Read the alignment results from the output file
alignment = AlignIO.read(output_file, "clustal")

# Print the alignment results
print(alignment)


In [ ]:
from Bio.Align.Applications import MafftCommandline
from Bio import AlignIO

mafft_exe = "/apps/mafft/7.505/bin/mafft"
input_file = path+ "/" + "Fukomys_damarensis.DMR_v1.0.dna.toplevel.fasta"

mafft_cline = MafftCommandline(mafft_exe, input = input_file)
print(mafft_cline)
stdout, stderr = mafft_cline()
print("finished mafft")
with open("aligned.fasta", "w") as handle:
    handle.write(stdout)
align = AlignIO.read("aligned.fasta", "fasta")

In [15]:
from Bio.Align.Applications import FProtDistCommandline

# Define the input and output file names
input_file = "sequences.fasta"
output_file = "aligned_sequences.phy"

cline = FProtDistCommandline(cmd='fprotdist', )
# Run Phylip to perform multiple sequence alignment
stdout, stderr = phylip_cline()

# Print the standard output and standard error messages
print(stdout)
print(stderr)


ImportError: cannot import name 'PhylipCommandline' from 'Bio.Align.Applications' (/home/ktrang/.local/lib/python3.10/site-packages/Bio/Align/Applications/__init__.py)